# Pour regrouper des achats par SCC (Strongly Connected Components) https://www.geeksforgeeks.org/strongly-connected-components/

Deniz Sert
12/06/23

In [26]:
from tqdm.notebook import tqdm as progressbar

In [1]:
import pandas as pd
import numpy as np

df_adjacency_list = pd.read_parquet('early_adj.parquet')


# Create dummy data points
# data = [
#     [1, 2, 100, 10],
#     [2, 1, 50, 15],
#     [3, 4, 200, 20],
#     [4, 5, 150, 25],
#     [5, 6, 300, 30],
#     [6, 5, 75, 35],
#     [7, 8, 250, 40],
#     [8, 9, 125, 45],
#     [9, 10, 400, 50],
#     [10, 11, 175, 55],
#     [11, 10, 225, 60],
#     [12, 13, 350, 65],
#     [13, 14, 275, 70],
#     [14, 15, 125, 75],
#     [15, 1, 275, 80],
#     [16, 17, 150, 85],
#     [17, 18, 200, 90],
#     [18, 15, 100, 95],
#     [19, 2, 400, 100],
#     [20, 1, 50, 105]
# ]

# Create a DataFrame with the dummy data
# df_dummy = pd.DataFrame(data, columns=["addr_id1", "addr_id2", "value", "height"]).astype(
#     {
#         "addr_id1": "UInt32",
#         "addr_id2": "UInt32",
#         "value": "UInt64",
#         "height": "UInt32",
#     }
# )

# Append the DataFrame to the adjacency list
# df_adjacency_list = df_dummy
# print(df_adjacency_list)

In [2]:
import networkx as nx
import matplotlib.pyplot as plt

def create_dag(adj_list):
  # init empty DAG
  dag = nx.DiGraph()

  for ix, entry in adj_list.iterrows():
      source = entry["addr_id1"]
      destination = entry["addr_id2"]
      value = entry["value"]
      height = entry["height"]
      dag.add_edge(source, destination, value=value, height=height)


  nx.draw_networkx(dag, with_labels=True, node_color='lightblue', edge_color='gray')
  plt.show()
  print(dag)

  return dag

ModuleNotFoundError: No module named 'networkx'

In [3]:
from collections import defaultdict

def cluster_nodes(adjacency_list):
    dag = create_directed_graph(adjacency_list)
    clusters = identify_clusters(dag)
    
    # Analyze clusters and identify specific characteristics
    
    return clusters

def create_directed_graph(adjacency_list):
    dag = defaultdict(list)
    for entry in adjacency_list:
        source = entry["addr_id1"]
        destination = entry["addr_id2"]
        value = entry["value"]
        height = entry["height"]
        dag[source].append((destination, value, height))
    
    return dag

def identify_clusters(dag):
    # Use Strongly Connected Components (SCC) algorithm
    # to identify clusters within the DAG
    clusters = []
    
    # Implement SCC algorithm here
    
    return clusters

In [ ]:
#  df_adjacency_list.append(
#              pd.DataFrame(
#                  adjacency, columns=["addr_id1", "addr_id2", "value",
# "height"]
#              ).astype(
#                  {
#                      "addr_id1": "UInt32",
#                      "addr_id2": "UInt32",
#                      "value": "UInt64",
#                      "height": "UInt32",
#                  }
#              )
#          )

# 1.5B unique address
# 10B tuples

## SCC peut été implementé par l’algorithme de Tarjan

In [29]:
import numpy as np

def tarjans_algorithm(graph):
    index_counter = [0]
    stack = []
    lowlink = {}
    indices = {}
    visited = set()
    result = []

    def strongconnect(node):
        indices[node] = index_counter[0]
        lowlink[node] = index_counter[0]
        index_counter[0] += 1
        stack.append(node)
        visited.add(node)

        successors = list(graph.loc[graph['addr_id1'] == node]['addr_id2'])
        for successor in successors:
            if successor not in visited:
                strongconnect(successor)
                lowlink[node] = min(lowlink[node], lowlink[successor])
            elif successor in stack:
                lowlink[node] = min(lowlink[node], indices[successor])

        if lowlink[node] == indices[node]:
            scc = []
            while True:
                successor = stack.pop()
                scc.append(successor)
                if pd.isnull(successor) or successor == node:
                    break
            result.append(scc)

    for node in progressbar(graph['addr_id1'].unique()):
        if node not in visited:
            strongconnect(node)

    return result


# Example usage
strongly_connected_components = tarjans_algorithm(df_adjacency_list[:1000])
print(strongly_connected_components)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html